# ReMKiT1D input generator - Poisson solver

This notebook creates the json input file for a poisson solver model.
The model being solved is:
$$
\nabla^2\phi = f
$$
Where $f$ is the charge distribution.

In [1]:
import numpy as np
import xarray as xr
import sys

import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib as mpl
from holoviews import opts
import panel as pn

sys.path.append('../')
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
from RMK_support import RKWrapper ,Grid
import RMK_support.sk_normalization as skn
import RMK_support.dashboard_support as ds

### Useful constants

In [2]:
elCharge = 1.60218e-19
elMass = 9.10938e-31
amu = 1.6605390666e-27  # atomic mass unit
ionMass = 2.014*amu  # deuterium mass
epsilon0 = 8.854188e-12  # vacuum permittivity
heavySpeciesMass = 2.014  # in amus

### Wrapper initialisation

In [3]:
rk = RKWrapper()

### Global parameters for IO files

In [4]:
rk.jsonFilepath = "./config.json"  # Default value
hdf5Filepath = "./RMKOutput/RMK_poisson/"
rk.setHDF5Path(hdf5Filepath)

### Setting options for external libraries used by ReMKiT1D

### MPI

In [5]:
numProcsX = 4  # Number of processes in x direction
numProcsH = 1  # Number of processes in harmonic
numProcs = numProcsX * numProcsH
haloWidth = 1  # Halo width in cells

rk.setMPIData(numProcsX, numProcsH, haloWidth)

### Normalisation

In [6]:
rk.setNormDensity(1.0e19)
rk.setNormTemperature(10.0)
rk.setNormRefZ(1.0)

# for convenience
tempNorm = rk.normalization["eVTemperature"]
densNorm = rk.normalization["density"]

norms = skn.calculateNorms(tempNorm,densNorm,1.0)

timeNorm = norms["time"]
sigmaNorm = norms["crossSection"]
lengthNorm = norms["length"]
speedNorm = norms["speed"]

### Grid initialisation

In [7]:
dx0 = 2*0.27/4
dxN = 2*0.0125/4
Nx = 64*4
L = 10 # Length in meters
xGridWidths = L/Nx*np.ones(Nx)
dv0 = 0.05
dvN = 0.4
Nv = 80
vGridWidths = np.geomspace(dv0, dvN, Nv)
lMax = 0
gridObj = Grid(xGridWidths, vGridWidths, lMax, interpretXGridAsWidths=True,
               interpretVGridAsWidths=True, isLengthInMeters=True)

In [8]:
# Add the grid to the config file
rk.grid = gridObj

### Creating initial charge density configuration

In [9]:
rho = np.zeros(Nx)
phi = np.ones(Nx)

### Creating variables

In [10]:
rk.addVar("phi", phi, isCommunicated=True, isStationary=True, units=r"$\epsilon_0/en_0x_0^2$")
rk.addVar("rho", rho, isCommunicated=True)

rk.addVar("dummy", np.ones(Nx))

rk.addVar("phiLeft",isDerived=True,isScalar=True)
rk.addVar("phiRight",isDerived=True,isScalar=True)

rk.addVar('time',isDerived=True,isScalar=True)

### Creating Poisson equation model

In [11]:
# Adding the model tag to tag list
modelTag = "poisson"

# Initializing model
poissonModel = sc.CustomModel(modelTag=modelTag)

evolvedVar = "phi"

# Adding first laplacian term from poisson's eq.
laplacian = sc.GeneralMatrixTerm(evolvedVar,customNormConst=-epsilon0/elCharge*densNorm*lengthNorm**2,stencilData=sc.diffusionStencil("none",[]))

poissonModel.addTerm("laplacian",laplacian)

# Adding charge distribution term
chargeDist = sc.GeneralMatrixTerm(evolvedVar,implicitVar="rho",stencilData=sc.diagonalStencil())

poissonModel.addTerm("chargeDist",chargeDist)

# Defining boundary terms

# Calculating
dxN = xGridWidths[-1]/lengthNorm

rightBCPhi = sc.GeneralMatrixTerm(evolvedVar,customNormConst=epsilon0/elCharge*densNorm*dxN**2,stencilData=sc.diagonalStencil([Nx]))

poissonModel.addTerm("rightBCPhi", rightBCPhi)

rightBCPhiRight = sc.GeneralMatrixTerm(evolvedVar,customNormConst=-epsilon0/elCharge*densNorm*dxN**2,stencilData=sc.diagonalStencil([Nx]),implicitVar="dummy")

poissonModel.addTerm("rightBCPhiRight", rightBCPhiRight)

dx0 = xGridWidths[0]/lengthNorm

leftBCPhi = sc.GeneralMatrixTerm(evolvedVar,customNormConst=-epsilon0/elCharge*densNorm*dx0**2,stencilData=sc.diagonalStencil([1]))

poissonModel.addTerm("leftBCPhi", leftBCPhi)

leftBCPhiLeft = sc.GeneralMatrixTerm(evolvedVar,customNormConst=0,stencilData=sc.diagonalStencil([1]),implicitVar="dummy")

poissonModel.addTerm("leftBCPhiLeft", leftBCPhiLeft)

rk.addModel(poissonModel.dict())

### Integrator options

In [12]:
integrator = sc.picardBDEIntegrator(absTol=10.0,convergenceVars=["phi"])

rk.addIntegrator("BE",integrator)

initialTimestep=0.1
rk.setIntegratorGlobalData(initialTimestep=initialTimestep)

bdeStep = sc.IntegrationStep("BE",defaultUpdateModelData=True)

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE",bdeStep.dict())

step = 1
out = 1


rk.setFixedNumTimesteps(step)
rk.setFixedStepOutput(out)

rk.writeConfigFile()

### Analysis

In [13]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=150,dpi=150)

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

In [14]:
numFiles = 1

loadFilenames = [hdf5Filepath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]

loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames)

loadedData

<xarray.Dataset>
Dimensions:   (x: 256, h: 1, v: 80, time: 2, dim_0: 1)
Coordinates:
  * x         (x) float64 0.01953 0.05859 0.09766 0.1367 ... 9.902 9.941 9.98
  * h         (h) int64 0
  * v         (v) float64 0.025 0.07567 0.1277 0.1811 ... 12.54 12.93 13.32
  * time      (time) float64 0.0 0.1
Dimensions without coordinates: dim_0
Data variables:
    phi       (time, x) float64 1.0 1.0 1.0 1.0 ... 0.9778 0.9818 0.9857 0.9896
    rho       (time, x) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    dummy     (time, x) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    phiLeft   (time, dim_0) float64 0.0 0.0
    phiRight  (time, dim_0) float64 0.0 0.0

In [15]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,gridObj)

dashboard.fluid2Comparison().show()

Launching server at http://localhost:32963
